# Recursion

So far, we have solved problems requiring an unbound number of steps (drawing a line of length $N$, where $N$ is given by input and thus unknown) with loops. Loops express very well "linear" processes, in which we accumulate the work done so far in the state, and at every iteration of the loop we simply add yet another "step" to the solution.

We will now focus on a comparable technique which, instead of using loops, makes use of functions in order to achieve the same result. Let us go back to a simple sample, a line of asterisks of length `n`, in order to have a problem to solve but without the excessive complexity of the problem getting in the way of our exploration.

Let us, once again, observe a line of six asterisks: `"******"`. Instead of splitting the line in a series of elementary elements, the steps, which are then assembled left-to-right (the individual asterisks), let us observe something else. A line can be built by concatenating other lines together! For example, the line above could be decomposed as:

- `"*" + "*****"`
- `"**" + "****"`
- `"***" + "***"`
- `"****" + "**"`
- `"*****" + "*"`

Let us focus on the very first of these decompositions: a line of length $n$ is made up of an asterisk, followed by another line of length $n-1$:

$$\underbrace{\texttt{"*...*"}}_{n} = \texttt{"*"} + \underbrace{\texttt{"*...*"}}_{n-1}$$

Of course, it does not really make much sense to define a line of negative length, which would be attempted by the definition above when $n=0$. In such cases, we skip the decomposition and we can simply produce an empty line, which happens to be the very same as an empty string:

$$\underbrace{\texttt{"*...*"}}_{0} = \texttt{""}$$


Suppose we were defining `line` as a function, with a single parameter `n`, then we could attempt writing the above formula in code, as follows:

```
line = n =>
  if n = 0 then
    return ""
  else
    rest = line(n-1)
    return "*" + rest
```

The code above looks odd, but not unreasonable. Even more surprisingly, translation into Python produces an actually working program!

In [1]:
def line(n):
    if n == 0:
        return ""
    else:
        rest = line(n-1)
        return "*" + rest
print(line(int(input())))

6
******


Let us now briefly study the state trace of this program:
[[STATE TRACE]]

The stack grows, creating one entry for each asterisk to be added. When one of the intermediate calls to `line` returns, then we add its value to the return expression, which grows "right-to-left" in a cascade of further `return` instructions which assemble the final line.

We have written our first _recursive_ function.

## Aggressive splitting
Of all the possible decompositions we have seen above, the one in the middle (`"***" + "***"`) is perhaps even more interesting than the one we have just implemented. The main attraction of this decomposition is its simmetry, that is a line of length $N$ can be thought of as the combination of two lines of length $N/2$.

Therefore, we can restate this new piece of knowledge about lines:

$$\underbrace{\texttt{"*...*"}}_{n} = \underbrace{\texttt{"*...*"}}_{n/2} + \underbrace{\texttt{"*...*"}}_{n/2}$$

Unfortunately, there is no real way to split an asterisk in two, so the above is understood as true only if $n$ is even. If $n$ were odd, then we would need an extra asterisk in the middle:

$$\underbrace{\texttt{"*...*"}}_{n} = \underbrace{\texttt{"*...*"}}_{n/2} + \texttt{"*"} + \underbrace{\texttt{"*...*"}}_{n/2}$$

The last, minor addition to our brief dissertation about the nature of lines is that for $n=0$ we get an empty line, which does not need to be decomposed into other lines:

$$\underbrace{\texttt{"*...*"}}_{0} = \texttt{""}$$

We could recap this formulation in a tighter format by writing as follows:

\begin{align*}
\underbrace{\texttt{"*...*"}}_{n} &= \texttt{""} & \text{ when } n=0 \\
&= \underbrace{\texttt{"*...*"}}_{n/2} + \underbrace{\texttt{"*...*"}}_{n/2} & \text{ when } n>0 \wedge n \text{ is even}  \\
&= \underbrace{\texttt{"*...*"}}_{n/2} + \texttt{"*"} + \underbrace{\texttt{"*...*"}}_{n/2} & \text{ when } n>0 \wedge n \text{ is odd}  \\
\end{align*}

This decomposition gets translated into a function quite directly: the checks to the right-hand-side of the equations above turn into one or more conditional statements. The smaller lines we decompose into have become a call from `line` to `line` itself. The resulting code is:

```
line = n =>
  if n = 0 then
    return ""
  else
    half = line(n//2)
    extra = if n % 2 = 0 then "" else "*"
    return half + half + extra
```

The translation into Python is straightforward and produces the following code:

In [4]:
def line(n):
    if n == 0:
        return ""
    else:
        half = line(n//2)
        extra = "" if n % 2 == 0 else "*" 
        return half + half + extra
print(line(int(input())))

7
*******


Let us now check the state trace:

[[STATE TRACE]]

Interestingly enough, this way of reasoning produces a significant improvement in our implementation of lines: now at every step we reduce the problem by a factor of two, so there are far less steps involved than in the previous implementations that added one single asterisk at a time!

## General idea of the process

The strategy we are using is called _recursion_, which is the programming equivalent of the mathematical principle of _induction_. This strategy follows a general template for solving problems:

1. First of all we state the problem. In our case: _drawing a line of `n` characters_.
- We then divide the problem into similar, smaller problems (the _inductive_ case). In our case: a line is made up of smaller lines.
- We identify a special case that is easy to identify and solve (the _base_ case). In our case: a line of $0$ characters is the empty string `""`.
- We now define a function to solve the problem: `line = n => ...`.
- The function checks the input to see if we are dealing with the base case or not. In our case, this becomes `if n = 0 then "" else ...`.
- In the `else` branch, we invoke the function recursively with adjusted parameters (**closer to the base case**). In our case:  `rest = line(n-1)`.
- Finally, we assemble the partial results in order to produce the final result. In our case: `return "*" + rest`.

Following the template we have just given, a recursive function will look as follows:

```
f = args =>
  if simple_to_solve then
    return base_solution
  else
    part1 := f(args1)
    part2 := f(args2)
    ...
    partN := f(argsN)
    return combine_parts
```

Again, when drawing lines recursively:
- `f` becomes `line`
- `args` becomes `n`
- `simple_to_solve` becomes `n = 0` (or `n <= 0` to also gracefully capture nonsensical calls such as `line(-3)`)
- `base_solution` becomes `""`
- `part1` becomes `"*"`
- `part2` becomes `rest`
- `f(args2)` becomes `line(n-1)`
- `combine_parts` becomes `part1 + part2`

## Power of recursion

Recursion is more difficult to get a grip on than loops. This is understandable, because whereas loops simply take steps in a sort of "forward" direction, recursion does something a bit more complex. When a recursive function invokes itself, then it generates a new instance of itself higher up in the stack. The calling function is not eliminated from the stack, but rather remains "dormant", waiting for its smaller instance to complete. When the smaller instance completes, then the caller gets resumed ("woken up") in order to complete its own process. This means that thinking about recursive functions must be done carefully, as they do not really match our intuition based on physical or geometric processes.

To justify this additional effort, an important question must be answered: is recursion more or less (or equally) powerful than loops? If recursion is just a more convoluted way of expressing loops, then there is no point to it. If it offers more expressive power on the other hand, then it might be worth the additional complexity.

We will now check whether or not loops can be implemented as recursive functions, and the other way around.

### Loops $\leq$ Recursion?

We begin our analysis by seeing whether or not loops can be emulated via recursive functions. Let us begin with a simple loop that sums the numbers from $1$ to $n$:

```
n := int(input())
sum := 0
while n > 0 do
  sum := sum + n
  n := n - 1
print(sum)
```

For this small program it is also quite easy to rephrase it in recursive terms, that is expressing the sum of numbers as a (shorter) sum of numbers:
- the sum of all numbers $1 \dots n$ is:
    - $0$ if $n = 0$
    - the sum of all numbers $1 \dots n-1$ plus $n$ otherwise

This leads us directly to the following recursive implementation:

```
sum_numbers_to = n =>
  if n <= 0 then
    return 0
  else
    rest := sum_numbers_to(n-1)
    return rest + n
    
sum := sum_numbers_to(int(input())
print(sum)
```

At a first glance, the two implementations we have given look slightly different, but it is clear that they share most of the key elements. In particular, let us isolate the key elements of the loop:

- the first line before the `while` loop, `sum := 0`, is the base case (`BASE`);
- the condition `n > 0` identifies when the base case _is not already met_ and we must proceed with further computation (`COND`);
- the body of the loop contains the argument to the recursive call `n-1` (`REC`);
- the body of the loop also contains the composition of the final result (`COMP`);

The loop can then be restated in terms of these elements as follows:

```
n := int(input())
sum := BASE
while COND do
  sum := COMP
  n := REC
``` 

The smaller constituent elements we have just identified are also present, in the exact same form (besides the condition, which is negated in the recursive version), in the recursive version. We can restate the recursive version in terms of these elements as follows:

```
f = n =>
  if not COND then
    return BASE
  else
    rest := f(REC)
    return COMP
```

This formulation of loops is very general, and it should be clear that it will likely cover most (if not all) loops we might want to build. The fact that we can always translate it into an equivalent recursive function strongly suggests that recursion is at least as powerful as loops. Moreover, we could even emulate `while` with a recursive function (we will see this in a later chapter)!

We can therefore conclude this first part of our comparison with the statement:

Loops $\leq$ Recursion

### Recursion $\leq$ Loops?

It is now crucial to understand whether or not the opposite is the case as well. If we can show that loops can be converted to recursive functions, but recursive functions can also be converted to loops, then it is clear that loops and recursion are essentially equivalent. If, on the other hand, we cannot show that recursive functions can be directly translated into loops, then we must conclude that recursion is, in some sense, more expressive than loops (there are many subtle caveats to this, but it is too soon to discuss them).

Let us take as an example of recursion the Fibonacci function, which models the growth of a population of rabbits in the wild and without predators:

```
fib = n => if n <= 1 then return n else return fib(n-1) + fib(n-2)
```

[[STATE TRACE]]

The stack and the program grow and change a lot, because of all the intermediate steps that must be frozen in place. At every level of the computation we double the number of steps with respect to the previous level.

Intuitively, translating this exact same problem with loops is impossible. The linear nature of loops ("take a step") is not capable of encoding the exponential size of the problem, at least without lots of additional machinery in place.

<div class="alert alert-block alert-info">
There are ways to implement the Fibonacci function with loops, of course. One way to do this would simply be to use dynamic programming, that is noticing that at each level we simply need to sum the two levels before, and doing this explicitly and bottom up solves the problem:


```
n := int(input())
n_1 := 0
n_2 := 1
while n > 0 do
  tmp = n_2 + n_1
  n_1 = n_2
  n_2 = tmp
  n = n - 1
```

Of course we are computing the same result, but the process is so different that effectively this is no translation of the original recursive function, but a completely different formulation altogether!

We might also notice that, if we added more complex tools to our toolbox, for example data structures such as an explicit stack, then we might simulate recursive functions with an explicit stack that models the waiting calls. This would, strictly speaking, implement a recursive function with a loop, but by explicitly emulating the way $\text{eval}$ and the stack in the state work. Therefore, we would consider this solution to be just an unwieldy implementation of recursion, and not a loop anymore, since the underlying mechanism would remain intact.
</div>


This shows us that recursion is indeed more powerful than loops, in the sense that loops can be directly, trivially reformulated into perfectly equivalent recursive functions that take the same steps in the same order. On the other hand, some complex recursive functions cannot be so easily turned back into loops.

In the following lectures we will take even more advantage of recursion to tackle more and more complex problems.